### Import Libraries


In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

### Auxiliary Functions

Here, a set of auxiliary functions is provided to assist in processing HTML tables


In [2]:
# Returns the data and time from the HTML table cell
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


# Returns the booster version from the HTML table cell
def booster_version(table_cells):
    out = "".join(
        [
            booster_version
            for i, booster_version in enumerate(table_cells.strings)
            if i % 2 == 0
        ][0:-1]
    )
    return out


# Returns the landing status from the HTML table cell
def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out


# Returns the mass from the HTML table cell
def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0 : mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


# Returns the landing status from the HTML table cell
def extract_column_from_header(row):
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = " ".join(row.contents)

    # Filter the digit and empty names
    if not (colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

### Request Data


In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

html_data = requests.get(url).text

soup = BeautifulSoup(html_data, "html5lib")

soup.find("title")

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Extract Data


In [4]:
# Find all soup object tables
html_tables = soup.find_all("table")

# Get third table
first_launch_table = html_tables[2]

# Get all columns
column_names = []
th_elements = first_launch_table.find_all("th")
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


### Create DataFrame


In [5]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict["Date and time ( )"]

launch_dict["Flight No."] = []
launch_dict["Launch site"] = []
launch_dict["Payload"] = []
launch_dict["Payload mass"] = []
launch_dict["Orbit"] = []
launch_dict["Customer"] = []
launch_dict["Launch outcome"] = []

# Added some new columns
launch_dict["Version Booster"] = []
launch_dict["Booster landing"] = []
launch_dict["Date"] = []
launch_dict["Time"] = []

In [6]:
extracted_row = 0

# Extract each table
for table_number, table in enumerate(
    soup.find_all("table", "wikitable plainrowheaders collapsible")
):
    # Get table row
    for rows in table.find_all("tr"):
        # Check to see if first table heading is a number corresponding to a launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # Get table element
        row = rows.find_all("td")

        # If it is a number save cells in a dictonary
        if flag:
            extracted_row += 1

            # Flight Number
            launch_dict["Flight No."].append(flight_number)

            # Date/Time
            datatimelist = date_time(row[0])

            date = datatimelist[0].strip(",")
            launch_dict["Date"].append(date)

            time = datatimelist[1]
            launch_dict["Time"].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not (bv):
                bv = row[1].a.string
            launch_dict["Version Booster"].append(bv)

            # Launch Site
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)

            # Payload
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)

            # Orbit
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)

            # Customer
            customer = row[6].a
            launch_dict["Customer"].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)

df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,[SpaceX],Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,[NASA],Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,[NASA],Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,[NASA],Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,[NASA],Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


### Export Web Scraped Data as CSV


In [7]:
df.to_csv("spacex_webscraped_dataset.csv", index=False)